# First step : loading data 

In [129]:
import pandas as pd
import random
import nltk

In [127]:
!sudo pip install nltk
!sudo python -m nltk.downloader all

Downloading/unpacking nltk
  Running setup.py (path:/tmp/pip_build_root/nltk/setup.py) egg_info for package nltk
    
  Running setup.py install for nltk
    
Cleaning up...
[nltk_data] Downloading collection u'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/ds/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /home/ds/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/ds/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /home/ds/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/ds/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /home/ds/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.z

In [130]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ds/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [131]:
nltk.download('corpora')

[nltk_data] Error loading corpora: Package 'corpora' not found in
[nltk_data]     index


False

In [157]:
data = pd.read_csv("data/cleaned.tsv", sep= "\t", header=None, names=["id", "sentiment", "text"])

In [158]:
data[data["sentiment"] == "positive"].count()

id           1334228
sentiment    1334228
text         1334228
dtype: int64

In [159]:
pos = data[data["sentiment"] == "positive"].sample(6800)

In [160]:
neg = data[data["sentiment"] == "negative"].sample(6800)

In [161]:
neu = data[data["sentiment"] == "neutral"].sample(6800)

In [162]:
print len(pos)
print len(neg)
print len(neu)

6800
6800
6800


In [179]:
from nltk import word_tokenize, WordNetLemmatizer

In [180]:
wordnet_lemmatizer = WordNetLemmatizer()

In [181]:
def preprocess(sentence):
    return [wordnet_lemmatizer.lemmatize(word.lower()) for word in word_tokenize(sentence.decode('utf-8'))]

In [164]:
dat = pd.concat([pos, neg, neu]).sample(frac=1)

Splitting the text by white spaces and punctuation marks – the tools that are used for this purpose are called tokenizers, and you can use a tokenizer provided with the NLTK.

In [165]:
dat.head()

,id,sentiment,text
1383809,790558958679302144,positive,@matthewlau18 @Jeraldawj trade for mb ;)
1825340,unavailable,neutral,@TheEconomist It appears Polls will return to...
826728,790153184236961792,positive,Post scene… Lovely bruises :) https://t.co/lCW...
1188372,790398248418553856,negative,All i want in life is to hug a panda :(
1720396,unavailable,neutral,griersunflower https://t.co/GJbCxZpZt1


In [173]:
tokenizer = lambda x: nltk.word_tokenize(x.decode('utf-8'))

In [182]:
dat["text"] = dat["text"].apply(preprocess)

In [184]:
dat.head()

,id,sentiment,text
1383809,790558958679302144,positive,"[@, matthewlau18, @, jeraldawj, trade, for, mb..."
1825340,unavailable,neutral,"[@, theeconomist, it, appears, poll, will, ret..."
826728,790153184236961792,positive,"[post, scene…, lovely, bruise, :, ), http, :, ..."
1188372,790398248418553856,negative,"[all, i, want, in, life, is, to, hug, a, panda..."
1720396,unavailable,neutral,"[griersunflower, http, :, //t.co/gjbcxzpzt1]"


linking the different forms of the same word (for example, price and prices, is and are) to each other – the tools that can do that are called lemmatizers, and you can again use one of those that come with the NLTK.

converting all words to lowercase so that the classifier does not treat People, people and PEOPLE as three separate features.

In [185]:
dat

,id,sentiment,text
1383809,790558958679302144,positive,"[@, matthewlau18, @, jeraldawj, trade, for, mb..."
1825340,unavailable,neutral,"[@, theeconomist, it, appears, poll, will, ret..."
826728,790153184236961792,positive,"[post, scene…, lovely, bruise, :, ), http, :, ..."
1188372,790398248418553856,negative,"[all, i, want, in, life, is, to, hug, a, panda..."
1720396,unavailable,neutral,"[griersunflower, http, :, //t.co/gjbcxzpzt1]"
537744,789904006978035713,positive,"[@, keplermessiah, @, maggyw519, @, ejlandwehr..."
1525166,790636482532958208,positive,"[@, everythingrobyn, @, seattlegators, we, 'd,..."
1704587,unavailable,neutral,"[happy, birthday, !, !, !, wish, u, many, more..."
69935,782720905566838784,positive,"[@, hh_kevriel91, thanks, for, welcoming, ,, k..."
1717237,unavailable,neutral,"[kai, is, not, a, good, guy, ., he, is, not, a..."


In [186]:
# Feature extraction

In [187]:
from nltk.corpus import stopwords

In [189]:
stoplist = stopwords.words("english")

In [191]:
twitter_stop_words = ["@", "rt"]

In [196]:
remove_stop_word = lambda list_of_words: [word for word in list_of_words if word not in stoplist+twitter_stop_words]

In [197]:
remove_stop_word(["a", "dog"])

['dog']

In [198]:
dat["text"] = dat["text"].apply(remove_stop_word)

In [201]:
from collections import Counter

In [202]:
def get_features(text, setting):
    if setting=='bow':
        return {word: count for word, count in Counter(preprocess(text)).items() if not word in stoplist}
    else:
        return {word: True for word in preprocess(text) if not word in stoplist}

In [ ]:
all_features = [(get_features(email, 'bow'), label) for (email, label) in all_emails]